In [1]:
import datetime
import requests
import json

Darksky key : 2b423d0c90e52bb805ccf6af1f305d7c ,bf11a6cc1e3fb105f612b086f6cefe19

In [7]:
def callApi(date):
    key = "bf11a6cc1e3fb105f612b086f6cefe19"
    latitude = "13.8282"
    longitude = "100.614"
    time = str(date).replace(" ", "T")
    url = "https://api.darksky.net/forecast/{}/{},{},{}".format(key, latitude, longitude, time)
    
    response = requests.get(url=url)
    return response

In [8]:
def one_month(month, year):
    date = datetime.datetime(year=year, month=month, day=21) #change day here
    nextMonth = date.month + 1
    while(date.month != nextMonth):
        yield date
        date = date + datetime.timedelta(days=1)   
        
def one_day(date):
    tomorrow = date + datetime.timedelta(days=1)
    while(date.day != tomorrow.day):
        yield date
        date = date + datetime.timedelta(minutes=15)   

In [4]:
def write_json(filename, data):
    with open("tongsdata/" + filename + ".json", "w", encoding='utf-8') as fout:
        fout.write(json.dumps(data))

___

In [5]:
year=2018
month=11
date = datetime.datetime(year=year, month=month, day=30)
nextMonth = date.month + 1
while(date.month != 1):
    date = date + datetime.timedelta(days=1)  
    print(date)

2018-12-01 00:00:00
2018-12-02 00:00:00
2018-12-03 00:00:00
2018-12-04 00:00:00
2018-12-05 00:00:00
2018-12-06 00:00:00
2018-12-07 00:00:00
2018-12-08 00:00:00
2018-12-09 00:00:00
2018-12-10 00:00:00
2018-12-11 00:00:00
2018-12-12 00:00:00
2018-12-13 00:00:00
2018-12-14 00:00:00
2018-12-15 00:00:00
2018-12-16 00:00:00
2018-12-17 00:00:00
2018-12-18 00:00:00
2018-12-19 00:00:00
2018-12-20 00:00:00
2018-12-21 00:00:00
2018-12-22 00:00:00
2018-12-23 00:00:00
2018-12-24 00:00:00
2018-12-25 00:00:00
2018-12-26 00:00:00
2018-12-27 00:00:00
2018-12-28 00:00:00
2018-12-29 00:00:00
2018-12-30 00:00:00
2018-12-31 00:00:00
2019-01-01 00:00:00


In [15]:
#  LOAD INTO FILES

for day in one_month(12, 2018): #change month here
    day_result = {}
    filename = day.strftime("%Y-%m-%d") #change to Y-m-d because file will arrange better
    for minute in one_day(day):
        result = callApi(minute)
        day_result.update({ minute.strftime("%H:%M"): result.json()})  #  key is time of the day
    write_json(filename=filename, data=day_result)  #  filename is date    

ValueError: Invalid format string

Don't forget to download Energybalance till the lastest day

___

## Clean Data

In [2]:
import os
import pandas as pd
import json
import datetime

In [3]:
def createDayDataframe(filename):
    path = "tongsdata/"
    filedate = filename.split(".")[0].split("-")

    with open(path + filename, 'r', encoding='utf-8') as fin:
        data_json = json.loads(fin.read())

    #  Create X DataFrame
    todayData = {time: data_json[time]['currently'] for time in data_json}
    todayData_DF = pd.DataFrame(todayData).T

    #  Create y DataFrame
    energyBalanceFilename = "energyBalance/Energy_Balance_{}_{}_{}.csv".format(filedate[0], filedate[1], filedate[2]) #change due to json filename
    energyBalance = pd.read_csv(path + energyBalanceFilename, delimiter=";")

    energyBalance_DF = pd.DataFrame(index=energyBalance.iloc[:-1,0].apply(lambda x: x[2:-1]))
    energyBalance_DF['PV Power Generation'] = energyBalance['PV power generation / Mean values [W]  '].values[:-1]
    energyBalance_DF = energyBalance_DF[energyBalance_DF != ' ']
    energyBalance_DF = energyBalance_DF.dropna()
    
    #  Join X y DataFrame
    data = todayData_DF.join(energyBalance_DF)
    return data

def cleanDataframe(masterDF):
    masterData_cleaned = masterDF.reset_index()
    masterData_cleaned = masterData_cleaned.rename(columns={'index': 'time_of_day'})
    masterData_cleaned = masterData_cleaned[masterData_cleaned['PV Power Generation'].notna()]
    masterData_cleaned['month'] = masterData_cleaned['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
    masterData_cleaned['cloudCover'] = masterData_cleaned['cloudCover'].fillna(value=masterData_cleaned['cloudCover'].mean())
    masterData_cleaned['PV Power Generation'] = masterData_cleaned['PV Power Generation'].apply(lambda x: x.replace(",", "")).astype('float')
    return masterData_cleaned

In [4]:
def featureEngineering(masterDF):
    masterDF = masterDF.drop(columns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure'])
    masterDF = pd.get_dummies(masterDF, columns=['time_of_day', 'precipType', 'uvIndex', 'month']) 
    return masterDF

In [5]:
files = os.listdir("tongsdata/")

#files.remove(".DS_Store")
files.remove("energyBalance")

masterData = pd.DataFrame()

for file in files:
    r = createDayDataframe(file).loc['06:30':'18:00']
    print(file, r.shape)
    masterData = pd.concat([masterData, r], axis=0)
    
masterData = cleanDataframe(masterData)
masterData = featureEngineering(masterData)

2018-11-01.json (47, 15)
2018-11-02.json (47, 13)
2018-11-03.json (47, 15)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


2018-11-04.json (47, 14)
2018-11-05.json (47, 14)
2018-11-06.json (47, 15)
2018-11-07.json (47, 14)
2018-11-08.json (47, 15)
2018-11-09.json (47, 14)
2018-11-10.json (47, 15)
2018-11-11.json (47, 14)
2018-11-12.json (47, 15)
2018-11-13.json (47, 14)
2018-11-14.json (47, 15)
2018-11-15.json (47, 15)
2018-11-16.json (47, 15)
2018-11-17.json (47, 14)
2018-11-18.json (47, 15)
2018-11-19.json (47, 15)
2018-11-20.json (47, 14)
2018-11-21.json (47, 15)
2018-11-22.json (47, 15)
2018-11-23.json (47, 15)
2018-11-24.json (47, 19)
2018-11-25.json (47, 19)
2018-11-26.json (47, 19)
2018-11-27.json (47, 19)
2018-11-28.json (47, 19)
2018-11-29.json (47, 15)
2018-11-30.json (47, 14)
2018-12-01.json (47, 15)
2018-12-02.json (47, 15)
2018-12-03.json (47, 14)
2018-12-04.json (47, 14)
2018-12-05.json (47, 14)
2018-12-06.json (47, 15)
2018-12-07.json (47, 14)
2018-12-08.json (47, 15)
2018-12-09.json (47, 15)
2018-12-10.json (47, 14)
2018-12-11.json (47, 14)
2018-12-12.json (47, 14)
2018-12-13.json (47, 14)


In [6]:
pd.set_option('display.max_columns', 100)
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2764 entries, 0 to 2819
Data columns (total 68 columns):
PV Power Generation    2764 non-null float64
apparentTemperature    2757 non-null object
cloudCover             2764 non-null float64
dewPoint               2757 non-null object
humidity               2757 non-null object
temperature            2757 non-null object
visibility             2756 non-null object
windSpeed              2756 non-null object
time_of_day_06:30      2764 non-null uint8
time_of_day_06:45      2764 non-null uint8
time_of_day_07:00      2764 non-null uint8
time_of_day_07:15      2764 non-null uint8
time_of_day_07:30      2764 non-null uint8
time_of_day_07:45      2764 non-null uint8
time_of_day_08:00      2764 non-null uint8
time_of_day_08:15      2764 non-null uint8
time_of_day_08:30      2764 non-null uint8
time_of_day_08:45      2764 non-null uint8
time_of_day_09:00      2764 non-null uint8
time_of_day_09:15      2764 non-null uint8
time_of_day_09:30      

In [6]:
masterData.shape

(2764, 68)

In [15]:
pd.set_option('display.max_columns', 100)
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1849 entries, 0 to 1879
Data columns (total 68 columns):
PV Power Generation    1849 non-null float64
apparentTemperature    1845 non-null object
cloudCover             1849 non-null float64
dewPoint               1845 non-null object
humidity               1845 non-null object
temperature            1845 non-null object
visibility             1844 non-null object
windSpeed              1844 non-null object
time_of_day_06:30      1849 non-null uint8
time_of_day_06:45      1849 non-null uint8
time_of_day_07:00      1849 non-null uint8
time_of_day_07:15      1849 non-null uint8
time_of_day_07:30      1849 non-null uint8
time_of_day_07:45      1849 non-null uint8
time_of_day_08:00      1849 non-null uint8
time_of_day_08:15      1849 non-null uint8
time_of_day_08:30      1849 non-null uint8
time_of_day_08:45      1849 non-null uint8
time_of_day_09:00      1849 non-null uint8
time_of_day_09:15      1849 non-null uint8
time_of_day_09:30      

Y
    * 'PV Power Generation'

X
    * 'time_of_day'         ->  one-hot
    * 'apparentTemperature' ->  OK
    * 'cloudCover'          ->  OK 
    * 'dewPoint'            ->  OK
    * 'humidity'            ->  OK
    * 'icon'                ->  drop
    * 'precipType'          ->  one-hot
    * 'summary'             ->  drop
    * 'temperature'         ->  OK
    * 'time'                ->  drop
    * 'uvIndex'             ->  one-hot
    * 'visibility'          ->  OK
    * 'windBearing'         ->  drop (NN NE EE ...)
    * 'windGust'            ->  drop (what is wind gust?)
    * 'windSpeed'           ->  OK
    * 'month'               ->  one-hot (Should do season or quarter?)
    * 'ozone'               ->  drop (why less data)
    * 'precipIntensity'     ->  drop (why less data)
    * 'precipProbability'   ->  drop (why less data)
    * 'pressure'            ->  drop (why less data)

___

In [7]:
features = list(masterData.drop(columns=['PV Power Generation']).columns)

In [8]:
masterData = masterData.dropna()

X = masterData.drop(columns=['PV Power Generation']).astype('float').values
y = masterData['PV Power Generation'].astype('float').values


In [7]:
masterData.to_csv("masterData_11.csv")

In [9]:
X.shape

(2756, 67)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR = LR.fit(X_train, y_train)

y_hat = LR.predict(X_test)

In [13]:
from sklearn.metrics import mean_squared_error, r2_score
import math

math.sqrt(mean_squared_error(y_test, y_hat))

367.6810834221374

In [13]:
result = pd.DataFrame()
result['Actual'] = y_test
result['Prediction'] = y_hat

In [17]:
result
result.to_csv("result2.csv")

In [12]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=10, population_size=10, verbosity=2) # more generation, pop size will make it closer,but it will take more time
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('best_model_60days.py')

Generation 1 - Current best internal CV score: -94383.07494257155
Generation 2 - Current best internal CV score: -94383.07494257155
Generation 3 - Current best internal CV score: -94383.07494257155
Generation 4 - Current best internal CV score: -94383.07494257155
Generation 5 - Current best internal CV score: -94383.07494257155
Generation 6 - Current best internal CV score: -89536.71170815293
Generation 7 - Current best internal CV score: -89536.71170815293
Generation 8 - Current best internal CV score: -89536.71170815293
Generation 9 - Current best internal CV score: -89536.71170815293
Generation 10 - Current best internal CV score: -89536.71170815293

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.5, min_samples_leaf=3, min_samples_split=2, n_estimators=100)
-81408.13360819068


True

model 3 RandomForestRegressor(only 1 estimator)

In [21]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-91867.4890210794
exported_pipeline = RandomForestRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=1, min_samples_split=7, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
print(mean_squared_error(results, testing_target), 
mean_absolute_error(results, testing_target), 
r2_score(results, testing_target))

95003.7092815859 209.76968674407354 0.8046446768254627


model 4 StackingEstimator(2 estimators) better mse, mae, r2 score

In [13]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator, ZeroCount

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-87019.59662711709
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=False, max_features=0.5, min_samples_leaf=3, min_samples_split=8, n_estimators=100)),
    ZeroCount(),
    RandomForestRegressor(bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=6, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
print(mean_squared_error(results, testing_target), 
mean_absolute_error(results, testing_target), 
r2_score(results, testing_target))

82411.68873688129 185.46956724303556 0.8431508099074713


### Model 60days

In [24]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-89536.71170815293
exported_pipeline = RandomForestRegressor(bootstrap=False, max_features=0.5, min_samples_leaf=3, min_samples_split=2, n_estimators=100)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
print(mean_squared_error(results, testing_target), 
      mean_absolute_error(results, testing_target), 
      r2_score(results, testing_target))

74366.30805027609 187.96794942085407 0.8438942551854669


In [19]:
from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
mean_squared_error(results, testing_target)
mean_absolute_error(results, testing_target)
#r2_score(results, testing_target)
#mean_absolute_percentage_error(testing_target, results)

213.94625504322767

In [20]:
#from sklearn.feature_selection import SelectFromModel
featuresImportance = pd.DataFrame()
featuresImportance['Feature Name'] = features
featuresImportance['Importance'] = exported_pipeline.feature_importances_

featuresImportance.sort_values('Importance', ascending=False)

,Feature Name,Importance
55,uvIndex_0,0.368590
3,humidity,0.129834
56,uvIndex_1,0.112739
4,temperature,0.060932
0,apparentTemperature,0.043527
2,dewPoint,0.042313
57,uvIndex_2,0.039707
1,cloudCover,0.035607
6,windSpeed,0.034908
5,visibility,0.024353


In [17]:
result = pd.DataFrame()
result['Actual'] = testing_target
result['Prediction'] = results
result['Error'] = np.abs(testing_target - results)
result

,Actual,Prediction,Error
0,845.0,661.618333,183.381667
1,482.0,582.109024,100.109024
2,176.0,214.320630,38.320630
3,491.0,486.884333,4.115667
4,1665.0,1743.065000,78.065000
5,8.0,36.085000,28.085000
6,2380.0,2207.388643,172.611357
7,2152.0,1956.791667,195.208333
8,1224.0,959.011935,264.988065
9,742.0,439.489762,302.510238


In [18]:
result.describe()

,Actual,Prediction,Error
count,689.000000,689.000000,689.000000
mean,1054.667634,1068.545370,192.845893
std,757.849590,712.836773,208.525721
min,0.000000,0.253500,0.209000
25%,315.000000,415.637571,44.890333
50%,1017.000000,1064.977500,122.535321
75%,1690.000000,1680.696429,261.950821
max,2806.000000,2481.511439,1418.835833


In [19]:
result = pd.DataFrame()
result['Actual'] = testing_target
result['Prediction'] = results
result['Error'] = np.abs(testing_target - results)
result.to_csv("result_tpot_60days.csv")

___

In [394]:
edaData = masterData

In [397]:
edaData['windSpeed']

0      3.21
1      3.21
2      3.25
3      3.25
4      3.24
5      3.25
6      3.25
7      2.93
8      3.28
9      4.13
10     5.25
11     5.44
12     5.62
13     5.81
14        6
15     6.73
16     7.46
17     8.19
18     8.92
19     8.75
20     8.65
21     8.61
22     8.66
23      8.7
24     8.78
25     8.89
26     9.04
27     8.78
28     8.52
29     8.27
       ... 
439    6.56
440    6.63
441    6.73
442    6.29
443    5.88
444    5.49
445    5.13
446    5.41
447     5.7
448       6
449     6.3
450    6.89
451    7.56
452    8.27
453    9.03
454    8.41
455    7.81
456    7.26
457    6.75
458    6.99
459    7.34
460    7.77
461    8.27
462    7.85
463    7.45
464    7.08
465    6.75
466    6.73
467    6.72
468    6.73
Name: windSpeed, Length: 465, dtype: object

TEST

In [21]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100